In [217]:
# Importing necessary libraries
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import Counter

# Define the number of qubits
n_qubits = 3
shots = 100
# Initialize a device
dev = qml.device('default.qubit', wires=n_qubits, shots=shots)

# Define the initial state preparation function
def initial_state():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])

# Define the weak error function
def apply_weak_error():
    qubit_to_flip = random.randint(0, 2)
    epsilon = np.pi / 100
    qml.RX(epsilon, wires=qubit_to_flip)


@qml.qnode(dev)
def circuit():
    initial_state()
    apply_weak_error()
    
    z1z2 = qml.sample(qml.PauliZ(0) @ qml.PauliZ(1))
    
    # Measure the stabilizers
    Z1Z2 = qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))
    Z1Z3 = qml.expval(qml.PauliZ(0) @ qml.PauliZ(2))
    
    # Define error correction based on the stabilizer measurements
    if Z1Z2 == -1 and Z1Z3 == -1:
        qml.PauliX(wires=0)
    elif Z1Z2 == -1 and Z1Z3 == 1:
        qml.PauliX(wires=1)
    elif Z1Z3 == 1 and Z1Z2 == -1:
        qml.PauliX(wires=2)

    corrected_state = qml.sample(wires=[0, 1, 2])
    
    # Return all measurements in the order they were taken
    return z1z2, Z1Z2, Z1Z3, corrected_state


# Run the circuit
z1z2, Z1Z2, Z1Z3, corrected_state = circuit()

# Print the stabilizer measurements
print('Z1Z2:', Z1Z2)
print('Z1Z3:', Z1Z3)

# Print the corrected state
print('Corrected state:', corrected_state)





Z1Z2: 1.0
Z1Z3: 1.0
Corrected state: [[0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [0 0 0]
 [1 1 1]]


AttributeError: 'numpy.ndarray' object has no attribute 'unique'